In [1]:
import numpy as np

<h1 style="color:gold"> A delicious Markov Chain </h1>

<img src="0.png" width="570" height="400">

## Burger : 0, Pizza : 1, Hotdog : 2

In [2]:
state = {
    0 : "Burger",
    1 : "Pizza",
    2 : "Hotdog"
}
state

{0: 'Burger', 1: 'Pizza', 2: 'Hotdog'}

<h1 style="color:gold">Transition Matrix</h1>
<img src="mat2.png" width="240" height="120">

## A_ij = P(X_n = j | X_n-1 = i)

In [3]:
A = np.array([[0.2, 0.6, 0.2], [0.3, 0.0, 0.7], [0.5, 0.0, 0.5]])
A

array([[0.2, 0.6, 0.2],
       [0.3, 0. , 0.7],
       [0.5, 0. , 0.5]])

In [4]:
A[0]

array([0.2, 0.6, 0.2])

In [5]:
np.random.choice( [0, 1, 2], p = A[0]) # returns a random number from 0, 1, 2 with probability A[0][0], A[0][1], A[0][2]

1

<h1 style="color:gold"> Random Walk on Markov Chain</h1>

In [6]:
n = 15
start_state = 0
curr_state = start_state
print(state[curr_state], "--->", end=" ")

while n-1:
    curr_state = np.random.choice([0, 1, 2], p=A[curr_state]) # Go to next state with probability A[curr_state]
    print(state[curr_state], "--->", end=" ")
    n-=1
print("stop")

Burger ---> Burger ---> Pizza ---> Hotdog ---> Burger ---> Burger ---> Pizza ---> Hotdog ---> Burger ---> Pizza ---> Hotdog ---> Burger ---> Hotdog ---> Burger ---> Pizza ---> stop


<h1 style="color:gold">Approach 1 : Monte Carlo</h1>

In [7]:
steps = 10**6
start_state = 0
curr_state = start_state
pi = np.array([0, 0, 0])
pi[start_state] = 1

i = 0
while i<steps:
    curr_state = np.random.choice([0,1,2], p=A[curr_state])
    pi[curr_state]+=1
    i +=1

print("π = ", pi/steps)

π =  [0.352254 0.211077 0.43667 ]


<h1 style="color:gold">Approach 2 : Repeated Matrix Multiplication</h1>

In [8]:
steps = 10**3
A_n = A

i=0
while i<steps:
    A_n =  np.matmul(A_n, A)
    i+=1

print("A^n = \n", A_n, "\n")
print("π = ", A_n[0])

A^n = 
 [[0.35211268 0.21126761 0.43661972]
 [0.35211268 0.21126761 0.43661972]
 [0.35211268 0.21126761 0.43661972]] 

π =  [0.35211268 0.21126761 0.43661972]


<h1 style="color:gold">Approach 3 : Finding Left Eigen Vectors</h1>

In [9]:
import scipy.linalg
values, left = scipy.linalg.eig(A, right = False, left = True) # solving pi*P = pi

print("left eigen vectors = \n", left, "\n")
print("eigen values = \n", values)

left eigen vectors = 
 [[-0.58746336+0.j         -0.16984156-0.35355339j -0.16984156+0.35355339j]
 [-0.35247801+0.j          0.67936622+0.j          0.67936622-0.j        ]
 [-0.72845456+0.j         -0.50952467+0.35355339j -0.50952467-0.35355339j]] 

eigen values = 
 [ 1.  +0.j        -0.15+0.3122499j -0.15-0.3122499j]


In [10]:
pi = left[:,0]
pi_normalized = [(x/np.sum(pi)).real for x in pi]
pi_normalized

[0.352112676056338, 0.21126760563380279, 0.43661971830985913]

<h1 style="color:gold">P(Pizza --> Hotdog --> Hotdog --> Burger) = ?</h1>
<br>

## => P(X_0 = Pizza, X_1 = Hotdog, X_2 = Hotdog, X_3 = Burger)
## => P(X_0 = Pizza)  P(X_1 = Hotdog | X_0 = Pizza)  P(X_2 = Hotdog | X_1 = Hotdog)  P(X_3 = Burger | X_2 = Hotdog)

In [11]:
pi_normalized


[0.352112676056338, 0.21126760563380279, 0.43661971830985913]

In [12]:
pi[0]

(-0.5874633581157707+0j)

In [13]:
def find_prob(seq, A, pi):
    start_state = seq[0]
    prob = pi[start_state]
    prev_state, curr_state = start_state, start_state
    for i in range(1, len(seq)):
        curr_state = seq[i]
        prob *= A[prev_state][curr_state]
        prev_state = curr_state
    return prob

print(find_prob([1, 2, 2, 0], A, pi_normalized))

0.036971830985915485
